In [6]:
# based on my repo usb_hid_controller it is possible to control the robot with an standard usb controller

from class_hid import Controller
import time

from time import sleep
import win32com.client
import time

# connect to the robot
eng = win32com.client.Dispatch("CAO.CaoEngine")
ctrl = eng.Workspaces(0).AddController("RC1", "CaoProv.DENSO.RC.X", "", "Server=192.168.0.1") 
Arm1 = ctrl.AddRobot("Arm1", "")
task1 = ctrl.AddTask("RobSlave", "")
task1.start(1) 

try:
    Arm1.Execute("Motor", [1,1])  # Motor aus
    print("Motor wurde erfolgreich ausgeschaltet.")
except Exception as e:
    print(f"Es ist ein Fehler aufgetreten: {e}")
    time.sleep(5)

    if "Timeout occurred" in str(e):
        print("Ein Timeout ist aufgetreten.")
    else:
        print("Ein anderer Fehler ist aufgetreten.")

Arm1.Execute("ExtSpeed", [5,5,5]) 
IO_25 = ctrl.AddVariable("IO25", "")
IO_26 = ctrl.AddVariable("IO26", "")
IO_27 = ctrl.AddVariable("IO27", "")
IO_28 = ctrl.AddVariable("IO28", "")

IO_25.Value = 1
IO_26.Value = 0
IO_27.Value = 1
IO_28.Value = 0

current_position = Arm1.AddVariable("@Current_Position","")
#print(current_position) #(680.2056884765625, -161.53814697265625, 308.7040710449219, 0.0, 0.0, 33.91001510620117, 1.0)

controller = Controller('XBOX Controller',"0x045e","0x028e")

controller.connect()
controller.set_flag(False)

while not controller.data_available():
    print("Warten auf Datenverfügbarkeit...")
    time.sleep(1)  

while True:
    data = controller.get_raw_data()
    if(data[11] == 4): # Exit 
        break
    _y = ((data[2]-128)/127)
    _x = ((data[4]-128)/127)    
    _z = ((data[8]-128)/127)    
    
    curpos = str(current_position)
    array = [float(x) for x in curpos.strip("()").split(",")]
 
    offset=2 # move in one cycle
    new_pos = f"@P P({array[0]+(_x*offset)}, {array[1]+(_y*offset)}, {array[2]+(-1*_z*offset)},  0.0, 0.0, 22.87002182006836, 1.0)"
    #print(new_pos)
    Arm1.Move(1,new_pos, "SPEED=80,NEXT")
    
    if(data[11] == 8):
        IO_25.Value = not IO_25.Value
        IO_26.Value = not IO_26.Value
    if(data[11] == 2):
        IO_27.Value = not IO_27.Value
        IO_28.Value = not IO_28.Value
        
    #print(data) # Controller raw data
    time.sleep(0.5)

print("End")

controller.close()

Arm1.Execute("Motor", [0,0]) 

Es ist ein Fehler aufgetreten: (-2147352567, 'Exception occurred.', (0, 'ROBOTalk.RbtlkEngine.1', 'Timeout occurred', None, 0, -2147481344), None)
Ein Timeout ist aufgetreten.
found device
Warten auf Datenverfügbarkeit...

Waiting for data...
Press any (system keyboard) key to stop...
End
Close connection
Verbindung zu XBOX Controller geschlossen.


In [ ]:
# USB 
y 8
b 2
a 1
x 4

12 linker button
hor rechts 12
hor links 28
vert down 20 up 4



In [ ]:
Arm1.Execute("Motor", [0,0]) #Motor aus